In [ ]:
import ogr
import subprocess

import geopandas as gpd

Reproject a shape file

In [ ]:
!ogr2ogr ../scratch/deelbekkens_wgs84 -t_srs "EPSG:4326" ../data/deelbekkens/Deelbekken.shp

In [ ]:
# TODO: add creation of subcat met geopandas!

Single clip

voorbeeld ahv -> http://www.gdal.org/gdalwarp.html

**DOEL**: 1 deelbekken van de nete uitsnijden uit de tif:

In [ ]:
import subprocess

inraster = '../scratch/NE1_50M_SR/NE1_50M_SR.tif'
outraster = inraster.replace('.tif', '{}.tif'.format("_out"))    
inshape = "../scratch/subcat.shp"
subprocess.call(['gdalwarp', inraster, outraster, '-cutline', inshape, 
                     '-crop_to_cutline', '-overwrite'])

**DOEL**: provincie op naam opgeven en die uitsnijden

In [ ]:
provincies = gpd.read_file("../scratch/provinciebel.shp")
provincies

In [ ]:
inraster = '../scratch/NE1_50M_SR/NE1_50M_SR.tif'
outraster = inraster.replace('.tif', '{}.tif'.format("_OostVlaanderen"))    
inshape = "../scratch/provinciebel.shp"
subprocess.call(['gdalwarp', inraster, outraster, '-cutline', inshape, 
                 '-cwhere', "PROVINCIE='OOST-VLAANDEREN'", 
                 '-crop_to_cutline', 
                 '-overwrite'])

Maar ik wil dat liever automatisch doen voor alle individuele provincies en voor elk van die een geotiff maken:

In [ ]:
import ogr

inraster = '../scratch/NE1_50M_SR/NE1_50M_SR.tif'
outraster = inraster.replace('.tif', '{}.tif'.format("_OostVlaanderen"))    
inshape = "../scratch/provinciebel.shp"

ds = ogr.Open(inshape)
lyr = ds.GetLayer(0)

lyr.ResetReading()
ft = lyr.GetNextFeature()

while ft:

    province_name = ft.GetFieldAsString('PROVINCIE')
    print(province_name)

    outraster = inraster.replace('.tif', '_%s.tif' % province_name.replace('-', '_'))    
    subprocess.call(['gdalwarp', inraster, outraster, '-cutline', inshape, 
                     '-crop_to_cutline', '-cwhere', "PROVINCIE='%s'" %province_name])

    ft = lyr.GetNextFeature()

ds = None

To gain from my efforts, a reusable function would be what I could really use